### Import required libraries

In [2]:
import pandas as pd
import numpy as np

## Import train and test set
#### properties
| Variable | Definition | Key |
| :------- | ---------: | --: |
| survival | Survival | 0 = No, 1 = Yes |
|pclass|Ticket class|1 = 1st, 2 = 2nd, 3 = 3rd|
|sex|Sex|
|Age|Age in years|
|sibsp|# of siblings / spouses aboard the Titanic|
|parch|# of parents / children aboard the Titanic|	
|ticket|Ticket number|
|fare|Passenger fare|	
|cabin|Cabin number|
|embarked|Port of Embarkation|

In [81]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [5]:
gender_submission = pd.read_csv('data/gender_submission.csv')

### Basic Feature Engineering with the Titanic Data

In [64]:
import string
def substrings_in_string(big_string, substrings):
    if not isinstance(big_string, str):
        return np.nan
    for substring in substrings:
        if big_string.find(substring) != -1:
            return substring
    print (big_string)
    return np.nan

In [79]:
def add_title(df):
    title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                        'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                        'Don', 'Jonkheer']
    df['Title'] = df['Name'].map(lambda x: substrings_in_string(x, title_list))
    
    def replace_titles(x):
        title = x['Title']
        if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
            return 'Mr'
        elif title in ['Countess', 'Mme']:
            return 'Mrs'
        elif title in ['Mlle', 'Ms']:
            return 'Miss'
        elif title == 'Dr':
            if x['Sex'] == 'Male':
                return 'Mr'
            else:
                return 'Mrs'
        else:
            return title

    df['Title'] = df.apply(replace_titles, axis=1)

In [82]:
def add_properties(df):
    #Turning cabin number into Deck
    cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
    df['Deck'] = df['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
    df['Age*Class'] = df['Age'] * df['Pclass']
    df['Family_Size'] = df['SibSp'] + df['Parch']
    df['Fare_Per_Person'] = df['Fare'] / (df['Family_Size']+1)